In [ ]:
from ipynb.fs.full.CommonFunctions import col_lagger,split_sequences
import pandas as pd
import numpy as np
import hydroeval as he
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.multioutput import MultiOutputRegressor
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import os
import warnings
import copy
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

In [ ]:
warnings.filterwarnings('ignore')


mpl.rcParams['axes.unicode_minus'] = False
plt.rcParams["font.family"] = 'Nanum Brush Script OTF'
mpl.rcParams['axes.unicode_minus'] = False
plt.rcParams["font.family"] = 'NanumGothic'
plt.rcParams['font.size'] = 12
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['axes.labelsize'] = 12
plt.style.use('ggplot')

CB91_Blue = '#2CBDFE'
CB91_Green = '#47DBCD'
CB91_Pink = '#F3A0F2'
CB91_Purple = '#9D2EC5'
CB91_Violet = '#661D98'
CB91_Amber = '#F5B14C'

color_list = [CB91_Blue, 
              CB91_Pink, 
              CB91_Green, 
              CB91_Amber,
              CB91_Purple, 
              CB91_Violet]

plt.rcParams['axes.prop_cycle'] = plt.cycler(color=color_list)

seed = 42
np.random.seed(seed)

In [ ]:
def col_lagger(df,x,num_lags):
    df2 = copy.deepcopy(df)
    for lag in range(1,num_lags+1):
        df2[x+'_'+str(lag)] = df2[x].shift(lag)
        
    return df2

In [ ]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out-1
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [ ]:
df = pd.read_csv('..//Processed_data//weather_dam_wrn.csv',encoding='cp949',index_col=[0])
df.index = pd.to_datetime(df.index)

In [ ]:
df[['강우량(mm)+1']] = df[['강우량(mm)']].shift(-1)
#df[['강우량(mm)+2']] = df[['강우량(mm)']].shift(-2)
df = col_lagger(df,'강우량(mm)',7)
df = col_lagger(df,'유입량(㎥/s)',7)
df = col_lagger(df,'최저기온(°C)',7)
df = col_lagger(df,'최고기온(°C)',7)
df = col_lagger(df,'평균 풍속(m/s)',7)
df = col_lagger(df,'호우경보',7)
df = col_lagger(df,'평균 상대습도(%)',7)
df = col_lagger(df,'합계 일사량(MJ/m2)',7)

In [ ]:
df = df.drop(columns=['최저기온(°C)', '최고기온(°C)', 
                      '평균 풍속(m/s)', '평균 상대습도(%)',
                      '합계 일사량(MJ/m2)','호우경보'])
df = df.dropna()

In [ ]:
X = df[[c for c in df.columns if c != '유입량(㎥/s)']].values
y = df[['유입량(㎥/s)']].values

data_stacked = np.hstack((X,y))

del X
del y
X,y = split_sequences(data_stacked,1,2)
train_length = int(df.shape[0]*0.8)

train_X , train_y = X[:train_length, :] , y[:train_length, :]
test_X , test_y = X[train_length:, :] , y[train_length:, :]

# Scaling X and Y

In [ ]:
scaler1x = StandardScaler()
scaler1y = StandardScaler()

#Scale X
train_X_prescaled = train_X.reshape(train_X.shape[0],-1)
train_X1 = scaler1x.fit_transform(train_X_prescaled).reshape(train_X_prescaled.shape[0],-1)#,train_X_prescaled.shape[-1])

test_X_prescaled = test_X.reshape(test_X.shape[0],-1)
test_X1 = scaler1x.transform(test_X_prescaled).reshape(test_X.shape[0],-1)#,test_X.shape[-1])

#Scale y
train_y_prescaled = train_y.reshape(train_y.shape[0],-1)
train_y1 = scaler1y.fit_transform(train_y_prescaled).reshape(train_y.shape[0],-1)#,train_y.shape[-1])

test_y_prescaled = test_y.reshape(test_y.shape[0],-1)
test_y1 = scaler1y.transform(test_y_prescaled).reshape(test_y.shape[0],-1)#,test_y.shape[-1])

In [ ]:
print(train_X1.shape)
print(test_X1.shape)
print(train_y1.shape)
print(test_y1.shape)

In [ ]:
rf = RandomForestRegressor()
gb = GradientBoostingRegressor()
mlp = MLPRegressor()
svr = SVR()



#WRAPPER for RF, GB, MLP
wrapper_rf = MultiOutputRegressor(rf)
wrapper_gb = MultiOutputRegressor(gb)
wrapper_mlp = MultiOutputRegressor(mlp)
wrapper_svr = MultiOutputRegressor(svr)

test_y1_inv = scaler1y.inverse_transform(test_y1)

#grid search (SVR)
svr_params = {
    'estimator__kernel':['linear','poly','rbf'],
    'estimator__degree':[2,3,4,5],
    'estimator__gamma':['scale','auto']
}

grid_cv_svr = GridSearchCV(wrapper_svr,svr_params,n_jobs=-3)

#grid search (Gradient Boosting)
gb_params = {
    'estimator__loss':['ls','lad','huber','quantile'],
    'estimator__learning_rate':[0.1,0.01,0.001],
    'estimator__n_estimators':[100,200,300],
    'estimator__criterion':['friedman_mse','mse','mae']
    
}

grid_cv_gb = GridSearchCV(wrapper_gb,gb_params,n_jobs=-3)

#grid search (Random Forest)
rf_params = {
    'estimator__n_estimators':[100,200,500],
    'estimator__criterion':['mse','mae'],
    'estimator__max_features':['auto','sqrt','log2']
}

grid_cv_rf = GridSearchCV(wrapper_rf,rf_params,n_jobs=-3)

#grid search (MLP)
mlp_params = {
    'estimator__hidden_layer_sizes':[(30,30,30),(50,50,50),(100,100,100)],
    'estimator__activation':['identity','logistic','tanh','relu'],
    'estimator__solver':['lbfgs','sgd','adam'],
    'estimator__batch_size':[32,64,128],
    'estimator__learning_rate':['constant','invscaling','adaptive'],
    'estimator__shuffle':[False]
}

grid_cv_mlp = GridSearchCV(wrapper_mlp,mlp_params,n_jobs=-3)

#MLP
mlp2 = grid_cv_mlp.fit(train_X1,train_y1)
mlp_pred = mlp2.predict(test_X1)
mlp_pred_inv = scaler1y.inverse_transform(mlp_pred)



In [ ]:
print('..... Multilayer Perceptron ..... ')
print('*'*40)
print(f'RMSE for first day: {mean_squared_error(test_y1_inv[:,0],mlp_pred_inv[:,0],squared=False)}')
print(f'MAE for first day: {mean_absolute_error(test_y1_inv[:,0],mlp_pred_inv[:,0])}\n')

print(f'RMSE 2일: {mean_squared_error(test_y1_inv[:,1],mlp_pred_inv[:,1],squared=False)}')
print(f'MAE 2일: {mean_absolute_error(test_y1_inv[:,1],mlp_pred_inv[:,1])}\n')

print(f'NSE : {he.nse(test_y1_inv,mlp_pred_inv)}\n')
print('*'*60)
print()

#Random Forest
rf2 = grid_cv_rf.fit(train_X1,train_y1)
rf_pred = rf2.predict(test_X1)
rf_pred_inv = scaler1y.inverse_transform(rf_pred)
print('.... Random Forest ....')
print('*'*40)
print(f'RMSE 1일: {mean_squared_error(test_y1_inv[:,0],rf_pred_inv[:,0],squared=False)}')
print(f'MAE 1일: {mean_absolute_error(test_y1_inv[:,0],rf_pred_inv[:,0])}\n')

print(f'RMSE 2일: {mean_squared_error(test_y1_inv[:,1],rf_pred_inv[:,1],squared=False)}')
print(f'MAE 2일: {mean_absolute_error(test_y1_inv[:,1],rf_pred_inv[:,1])}\n')

print(f'NSE : {he.nse(test_y1_inv,rf_pred_inv)}\n')
print('*'*60)
print()

#Gradient Boosting
gb2 = grid_cv_gb.fit(train_X1,train_y1)
gb_pred = gb2.predict(test_X1)
gb_pred_inv = scaler1y.inverse_transform(gb_pred)
print('...Gradient Boosting ....')
print('*'*40)
print(f'RMSE 1일: {mean_squared_error(test_y1_inv[:,0],gb_pred_inv[:,0],squared=False)}')
print(f'MAE 1일: {mean_absolute_error(test_y1_inv[:,0],gb_pred_inv[:,0])}\n')

print(f'RMSE 2일: {mean_squared_error(test_y1_inv[:,1],gb_pred_inv[:,1],squared=False)}')
print(f'MAE 2일: {mean_absolute_error(test_y1_inv[:,1],gb_pred_inv[:,1])}\n')

print(f'NSE : {he.nse(test_y1_inv,gb_pred_inv)}\n')

print('*'*60)
print()

#Support Vector Regressor
svr2 = grid_cv_svr.fit(train_X1,train_y1)
svr_pred = svr2.predict(test_X1)
svr_pred_inv = scaler1y.inverse_transform(svr_pred)
print(f'.... Support Vector Regressor ....')
print('*'*40)
print(f'RMSE 1일: {mean_squared_error(test_y1_inv[:,0],svr_pred_inv[:,0],squared=False)}')
print(f'MAE 1일: {mean_absolute_error(test_y1_inv[:,0],svr_pred_inv[:,0])}')

print(f'RMSE 2일: {mean_squared_error(test_y1_inv[:,1],svr_pred_inv[:,1],squared=False)}')
print(f'MAE 2일: {mean_absolute_error(test_y1_inv[:,1],svr_pred_inv[:,1])}\n')

print(f'NSE : {he.nse(test_y1_inv,svr_pred_inv)}\n')

print()

In [ ]:
print('Best parameter for MLP')
print(grid_cv_mlp.best_params_)
print('*'*60)
print()
print('Best parameter for Gradient Boosting')
print(grid_cv_gb.best_params_)
print('*'*60)
print()
print('Best parameter for Random Forest')
print(grid_cv_rf.best_params_)
print('*'*60)
print()
print('Best parameter for Support Vector Regressor')
print(grid_cv_svr.best_params_)
print('*'*60)
print()